In [41]:
Z.<X> = ZZ[]
F = GF(3*5*2^27+1)

In [20]:
def mask(n_bits):
    return (1 << n_bits) - 1
def decomp(n, n_limbs, limb_bits):
    p = 0
    for i in range(n_limbs):
        p += X^i * (n & mask(limb_bits))
        n >>= limb_bits
    return p
p = decomp(12345678, n_limbs=4, limb_bits=8); print(p)
assert p(2^8) == 12345678

188*X^2 + 97*X + 78


hmm, you can either check the poly mod 2^32, or do a widening mul

In [54]:
# check the poly % 2^32
LIMBS = dict(n_limbs=4, limb_bits=8)
a, b = [randint(0, 2^32-1) for _ in range(2)]
c = (a * b) % 2^32
print(a, b, c)
f = decomp(a, **LIMBS) * decomp(b, **LIMBS) - decomp(c, **LIMBS)
print(f)
print(f(2^8))
# yes % 2^32 here
assert f(2^8) % 2^32 == 0

3811512014 4212560312 820597264
56977*X^6 + 16791*X^5 + 37463*X^4 + 100549*X^3 + 14977*X^2 + 32364*X + 37888
16056224238066991104


this doesnt seem to work:

In [55]:
f.change_ring(F)(2^8)

480504017

In [81]:
# c has twice as many limbs
N_LIMBS = 4
LIMB_BITS = 32
LIMBS = dict(n_limbs=N_LIMBS, limb_bits=LIMB_BITS)
a, b = [randint(0, 2^(N_LIMBS*LIMB_BITS)-1) for _ in range(2)]
c = a * b
print(a, b, c)
f = decomp(a, **LIMBS) * decomp(b, **LIMBS) - decomp(c, n_limbs=N_LIMBS*2, limb_bits=LIMB_BITS)
print(f)
print(f(2^LIMB_BITS))
# no % 2^32 here
assert f(2^LIMB_BITS) == 0

261505827213054352438808662556115414771 120148806299499634817305544703949791275 31419612980011687811533229999726740819910551628540678626237144626855501923025
-1165418217*X^7 + 5005433125944513899*X^6 + 9591165911194570265*X^5 + 9325109733864827798*X^4 + 28062796145761574721*X^3 + 12082630851008796887*X^2 + 17109710619884152058*X + 11822963896849793024
0


this does work:

In [83]:
f.change_ring(F)(2^LIMB_BITS)

0

In [65]:
# do a probabilistic check
# after committing a,b,c, challenge alpha:
alpha = F.random_element()
# verifier computes this quotient with constraints
quotient = (f(X) - f(2^8))/(X - 2^8); print(quotient)
quotient(alpha)

-77*X^6 - 129*X^5 - 200*X^4 - 276*X^3 - 190*X^2 - 153*X - 75


1514173140